In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime, date

In [ ]:
#Loading the data
tran = pd.read_excel("Raw_data.xlsx", sheet_name = "Transactions")

In [ ]:
#checking the 1st five records
tran.head(5)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [ ]:
#Checking the information about data
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

Observations:

*   Data is missing in couple of columns, need to resolve this

*   Check Duplicate data

*   Data type of "product first sold date" is not in date but seeing the values inside column doesn't look like a date so we will see it later



In [ ]:
tran.shape

(20000, 13)

In [ ]:
#Let's bifurcate the numerical and categorical data columns as well
tran_num_cols = tran.select_dtypes(include = "number")
print("Numerical Columns are: ",tran_num_cols.columns)

tran_cat_cols = tran.select_dtypes(exclude = "number")
print("Categorical Columns are: ",tran_cat_cols.columns)

Numerical Columns are:  Index(['transaction_id', 'product_id', 'customer_id', 'online_order',
       'list_price', 'standard_cost', 'product_first_sold_date'],
      dtype='object')
Categorical Columns are:  Index(['transaction_date', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size'],
      dtype='object')


## Checking missing values

In [ ]:
tran.isnull()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,False,False,False,False,False,False,False,False,False,False,False,False,False
19996,False,False,False,False,False,False,False,False,False,False,False,False,False
19997,False,False,False,False,False,False,False,False,False,False,False,False,False
19998,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
tran.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [ ]:
#percente of missing values against the whole data
tran.isnull().mean()*100

transaction_id             0.000
product_id                 0.000
customer_id                0.000
transaction_date           0.000
online_order               1.800
order_status               0.000
brand                      0.985
product_line               0.985
product_class              0.985
product_size               0.985
list_price                 0.000
standard_cost              0.985
product_first_sold_date    0.985
dtype: float64

So, we have 360 missing values in online_order column and 197 values missing in 5 of the columns. Let's start with "online_order" column.

"online_order" column comprises of 1.8% of the data so can not remove this, need to fill the missing values

while other columns as "brand", "product_line", "product_class","product_size","standard_cost","product_first_sold_date", comprises of 1% of the data, so can be removed

In [ ]:
tran["online_order"]

0        0.0
1        1.0
2        0.0
3        0.0
4        1.0
        ... 
19995    1.0
19996    1.0
19997    1.0
19998    0.0
19999    1.0
Name: online_order, Length: 20000, dtype: float64

In [ ]:
tran["online_order"].unique()

array([ 0.,  1., nan])

In [ ]:
#now there are 2 values which are in the column, either 0 or 1.0, we will fill the missing values with the mode value
mode_tran_online_order = tran["online_order"].mode()
mode_tran_online_order

0    1.0
Name: online_order, dtype: float64

In [ ]:
#now fill the missing data with 1
tran["online_order"] = tran["online_order"].fillna(1.0)

In [ ]:
#check if the missing values has been filled or not
tran["online_order"].isnull().sum()

0

In [ ]:
#let's check the complete columns once
tran.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order                 0
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [ ]:
tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             20000 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

Now, need to drop the NaN values from brand columng along with the other columns.

In [ ]:
tran[tran["brand"].isnull()]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
136,137,0,431,2017-09-23,0.0,Approved,NaN,NaN,NaN,NaN,1942.61,NaN,NaN
159,160,0,3300,2017-08-27,0.0,Approved,NaN,NaN,NaN,NaN,1656.86,NaN,NaN
366,367,0,1614,2017-03-10,0.0,Approved,NaN,NaN,NaN,NaN,850.89,NaN,NaN
406,407,0,2559,2017-06-14,1.0,Approved,NaN,NaN,NaN,NaN,710.59,NaN,NaN
676,677,0,2609,2017-07-02,0.0,Approved,NaN,NaN,NaN,NaN,1972.01,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19340,19341,0,443,2017-12-26,1.0,Approved,NaN,NaN,NaN,NaN,744.54,NaN,NaN
19383,19384,0,2407,2017-06-11,0.0,Approved,NaN,NaN,NaN,NaN,1098.18,NaN,NaN
19793,19794,0,2860,2017-01-13,0.0,Approved,NaN,NaN,NaN,NaN,868.56,NaN,NaN
19859,19860,0,2468,2017-06-24,1.0,Approved,NaN,NaN,NaN,NaN,1497.43,NaN,NaN


In [ ]:
tran[tran["brand"].isnull()][["brand", "product_line", "product_class", "product_size", "standard_cost", "product_first_sold_date"]]

,brand,product_line,product_class,product_size,standard_cost,product_first_sold_date
136,NaN,NaN,NaN,NaN,NaN,NaN
159,NaN,NaN,NaN,NaN,NaN,NaN
366,NaN,NaN,NaN,NaN,NaN,NaN
406,NaN,NaN,NaN,NaN,NaN,NaN
676,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
19340,NaN,NaN,NaN,NaN,NaN,NaN
19383,NaN,NaN,NaN,NaN,NaN,NaN
19793,NaN,NaN,NaN,NaN,NaN,NaN
19859,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tran[tran["brand"].isnull()][["brand", "product_line", "product_class", "product_size", "standard_cost", "product_first_sold_date"]].drop_duplicates()

,brand,product_line,product_class,product_size,standard_cost,product_first_sold_date
136,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
value_to_remove = tran[tran["brand"].isnull()][["brand", "product_line", "product_class", "product_size", "standard_cost", "product_first_sold_date"]].index

In [ ]:
tran.drop(value_to_remove, axis = 0, inplace = True)

In [ ]:
tran.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19803 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           19803 non-null  int64         
 1   product_id               19803 non-null  int64         
 2   customer_id              19803 non-null  int64         
 3   transaction_date         19803 non-null  datetime64[ns]
 4   online_order             19803 non-null  float64       
 5   order_status             19803 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               19803 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

In [ ]:
#all the missing values are now deleted.

## Checking if any data is not consistent

In [ ]:
  tran["online_order"].value_counts()

1.0    10097
0.0     9706
Name: online_order, dtype: int64

In [ ]:
  tran["order_status"].value_counts()

Approved     19625
Cancelled      178
Name: order_status, dtype: int64

In [ ]:
  tran["brand"].value_counts()

Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: brand, dtype: int64

In [ ]:
  tran["product_line"].value_counts()

Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: product_line, dtype: int64

In [ ]:
  tran["product_class"].value_counts()

medium    13826
high       3013
low        2964
Name: product_class, dtype: int64

In [ ]:
  tran["product_size"].value_counts()

medium    12990
large      3976
small      2837
Name: product_size, dtype: int64

## Checking duplicate data

In [ ]:
tran.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
19995    False
19996    False
19997    False
19998    False
19999    False
Length: 19803, dtype: bool

In [ ]:
tran[tran["transaction_id"].duplicated()]

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date


## Seems like there is not any duplicate entry, so the data is clean now and ready to export for visualization

In [ ]:
## Need to create a profit columns as well
tran["Profit"] = tran["list_price"] - tran["standard_cost"]

In [ ]:
tran.head(3)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,Profit
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0,17.87
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0,1702.55
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0,1544.61


In [ ]:
tran.to_csv("Transaction_cleaned.csv", index = False)